# Load necessary packages

In [ ]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

from entsoe import EntsoePandasClient

In [ ]:
features = pd.read_pickle(f"./features.pkl")

ID3 = pd.DataFrame()

ID3['ID3'] = features['ID3']

In [ ]:
ID3['ID3']

In [ ]:
features.tail(5)

In [ ]:
# def ts_train_test(ID3, time_steps, for_periods):

#     # Create training and test sets
#     mask = (ID3.index.year < 2018)
#     ts_train = ID3.loc[mask]['ID3'].values
#     ts_train = ts_train.reshape(1, -1).transpose()

#     mask = (ID3.index.year == 2018)
#     ts_test = ID3.loc[mask]['ID3'].values
#     ts_test = ts_test.reshape(1, -1).transpose()

#     # ts_train = ID3[:'2018'].iloc['ID3'].values
#     # ts_test  = ID3['2019':].iloc['ID3'].values

#     # Scale the data
#     from sklearn.preprocessing import MinMaxScaler
#     sc = MinMaxScaler(feature_range=(0,1))
#     ts_train_unscaled = ts_train
#     ts_train = sc.fit_transform(ts_train)

#     # Create training data of s samples and t time steps
#     X_train = []
#     y_train = []
#     y_train_stacked = []
    
#     for i in range(time_steps, len(ts_train)-1): 
#         X_train.append(ts_train[i-time_steps:i,0])
#         y_train.append(ts_train[i:i+for_periods,0])
#     X_train, y_train = np.array(X_train), np.array(y_train)

#     # Reshape X_train for efficient modelling
#     X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

#     # Prepare to create X_test
#     inputs = pd.concat((ID3['ID3'][:'2018'], ID3['ID3']['2019':]),axis=0).values
#     inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
#     inputs = inputs.reshape(-1,1)
#     inputs  = sc.transform(inputs)

#     X_test = []
#     for i in range(time_steps, len(ts_test)+time_steps-for_periods):
#         X_test.append(inputs[i-time_steps:i,0])
        
#     X_test = np.array(X_test)
#     X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

#     return X_train, y_train , X_test, sc

# ##########################################################################
# X_train, y_train, X_test, sc = ts_train_test(ID3, 5, 2)
# ##########################################################################

# X_train.shape[0], X_train.shape[1]

# print(X_train)

In [ ]:
time_steps = 5
for_periods = 1

lag = 4

# Create training and test sets
mask = (ID3.index.year < 2018)
ts_train = ID3.loc[mask]['ID3'].values
ts_train = ts_train.reshape(1, -1).transpose()

mask = (ID3.index.year == 2018)
ts_test = ID3.loc[mask]['ID3'].values
ts_test = ts_test.reshape(1, -1).transpose()

# Scale the data
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))

ts_train_unscaled = ts_train
ts_train = sc.fit_transform(ts_train)

# Create training data of s samples and t time steps
X_train = []
y_train = []
y_train_stacked = []

for i in range(time_steps+lag, len(ts_train)-1): 
    X_train.append(ts_train[i-lag-time_steps:i-lag,0])
    y_train.append(ts_train[i:i+for_periods,0])
X_train, y_train = np.array(X_train), np.array(y_train)

print(X_train)

# Reshape X_train for efficient modelling
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Prepare to create X_test
inputs = pd.concat((ID3['ID3'][:'2018'], ID3['ID3']['2019':]),axis=0).values
inputs = inputs[len(inputs)-len(ts_test) - time_steps:]
inputs = inputs.reshape(-1,1)
inputs  = sc.transform(inputs)

X_test = []
y_test = []

for i in range(time_steps+lag, len(ts_test)+time_steps-for_periods):
    X_test.append(inputs[i-lag-time_steps:i-lag,0])
    y_test.append(ts_test[i:i+for_periods,0])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

X_train.shape[0], X_train.shape[1]

print(X_train)
print(y_train)

In [ ]:
# inputs = features[['ID3']]
# outputs = features[['ID3']]

# inputs_train = inputs[inputs.index.year < 2018]
# outputs_train = outputs[outputs.index.year < 2018]

# inputs_test = inputs[inputs.index.year == 2018]
# outputs_test = outputs[outputs.index.year == 2018]

# # Scale inputs
# scaler = MinMaxScaler()

# inputs_train = pd.DataFrame(scaler.fit_transform(inputs_train), columns = inputs_train.columns)
# inputs_test = pd.DataFrame(scaler.transform(inputs_test), columns = inputs_test.columns)

# # Scale outputs
# sc = MinMaxScaler()

# sc.fit_transform(outputs_train)

# outputs_train = pd.DataFrame(sc.fit_transform(outputs_train), columns = outputs_train.columns)
# outputs_test = pd.DataFrame(sc.transform(outputs_test), columns = outputs_test.columns)

# time_steps = 6
# for_periods = 1
# lag = 4

# # X_train and y_train

# X_train = []
# y_train = []

# for i in range(0, len(outputs_train)-time_steps-lag+1):
#     for ii in range(0, time_steps):
#         X_train.extend(inputs_train.iloc[i+ii].to_numpy())
#     y_train.extend(outputs_train.iloc[i+time_steps+lag-1].to_numpy())

# X_train, y_train = np.array(X_train), np.array(y_train)

# X_train = X_train.reshape(y_train.shape[0], time_steps, inputs.shape[1])

# # X_test and y_test

# X_test = []
# y_test = []

# for i in range(0, len(outputs_test)-time_steps-lag+1):
#     for ii in range(0, time_steps):
#         X_test.extend(inputs_test.iloc[i+ii].to_numpy())
#     y_test.extend(outputs_test.iloc[i+time_steps+lag-1].to_numpy())

# X_test, y_test = np.array(X_test), np.array(y_test)

# sc = MinMaxScaler(feature_range=(0,1))

# X_test = X_test.reshape(y_test.shape[0], time_steps, inputs.shape[1])

In [ ]:
# sc.fit_transform(outputs_train)

In [ ]:
X_train.shape[0], X_train.shape[1]

In [ ]:
# Convert the 3-D shape of X_train to a data frame so we can see: 
X_train_see = pd.DataFrame(np.reshape(X_train, (X_train.shape[0], X_train.shape[1])))
y_train_see = pd.DataFrame(y_train)
pd.concat([X_train_see, y_train_see], axis=1)

# Convert the 3-D shape of X_test to a data frame so we can see: 
X_test_see = pd.DataFrame(np.reshape(X_test, (X_test.shape[0], X_test.shape[1])))
pd.DataFrame(X_test_see)

print(), print(colored(f'There are {X_train.shape[0]} samples in the training data', 'green'))
print(colored(f'There are {X_test.shape[0]} samples in the test data', 'green')), print()

print(X_train_see.head(20))
print(y_train_see.head(20))

In [ ]:
import keras
import keras.backend as K

class Dropout(keras.layers.Dropout):
  """Applies Dropout to the input.
  The Dropout layer randomly sets input units to 0 with a frequency of `rate`
  at each step during training time, which helps prevent overfitting.
  Inputs not set to 0 are scaled up by 1/(1 - rate) such that the sum over
  all inputs is unchanged.
  Note that the Dropout layer only applies when `training` is set to True
  such that no values are dropped during inference. When using `model.fit`,
  `training` will be appropriately set to True automatically, and in other
  contexts, you can set the kwarg explicitly to True when calling the layer.
  (This is in contrast to setting `trainable=False` for a Dropout layer.
  `trainable` does not affect the layer's behavior, as Dropout does
  not have any variables/weights that can be frozen during training.)
  >>> tf.random.set_seed(0)
  >>> layer = tf.keras.layers.Dropout(.2, input_shape=(2,))
  >>> data = np.arange(10).reshape(5, 2).astype(np.float32)
  >>> print(data)
  [[0. 1.]
   [2. 3.]
   [4. 5.]
   [6. 7.]
   [8. 9.]]
  >>> outputs = layer(data, training=True)
  >>> print(outputs)
  tf.Tensor(
  [[ 0.    1.25]
   [ 2.5   3.75]
   [ 5.    6.25]
   [ 7.5   8.75]
   [10.    0.  ]], shape=(5, 2), dtype=float32)
  Args:
    rate: Float between 0 and 1. Fraction of the input units to drop.
    noise_shape: 1D integer tensor representing the shape of the
      binary dropout mask that will be multiplied with the input.
      For instance, if your inputs have shape
      `(batch_size, timesteps, features)` and
      you want the dropout mask to be the same for all timesteps,
      you can use `noise_shape=(batch_size, 1, features)`.
    seed: A Python integer to use as random seed.
  Call arguments:
    inputs: Input tensor (of any rank).
    training: Python boolean indicating whether the layer should behave in
      training mode (adding dropout) or in inference mode (doing nothing).
  """
  
  def __init__(self, rate, training=None, noise_shape=None, seed=None, **kwargs):
    super(Dropout, self).__init__(rate, noise_shape=None, seed=None, **kwargs)
    self.training = training

  def call(self, inputs, training=None):
    if 0. < self.rate < 1.:
        noise_shape = self._get_noise_shape(inputs)

        def dropped_inputs():
            return K.dropout(inputs, self.rate, noise_shape, seed=self.seed)

        if not training:
            return K.in_train_phase(dropped_inputs, inputs, training=self.training)
        return K.in_train_phase(dropped_inputs, inputs, training=training)
    return inputs

In [ ]:
def LSTM_model(X_train, y_train, X_test, sc):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, GRU, LSTM
    from keras.optimizers import SGD, Adam
    
    model_lstm = Sequential()
    model_lstm.add(LSTM(units = 200, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
    model_lstm.add(LSTM(units = 100, return_sequences = True))
    model_lstm.add(LSTM(units = 50, return_sequences = False))
    
    model_lstm.add(Dense(units = 1, activation='linear'))

    optimizer = Adam(clipvalue=0.5)
    # model_lstm.compile(loss = 'mse', optimizer=optimizer)

    # Compiling
    model_lstm.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    model_lstm.fit(X_train,y_train,epochs=1,batch_size=32, verbose=1)

    LSTM_prediction = model_lstm.predict(X_test)
    LSTM_prediction = sc.inverse_transform(LSTM_prediction)

    # self.model = model_lstm

    return model_lstm, LSTM_prediction

def LSTM_model2(X_train, y_train, X_test, sc):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense, SimpleRNN, GRU, LSTM
    from keras.optimizers import SGD, Adam
    
    model_lstm = Sequential()
    
    model_lstm.add(LSTM(units = 200, return_sequences = True, input_shape=(X_train.shape[1],1), activation='tanh'))
    
    model_lstm.add(Dropout(0.2, training=True))
    
    model_lstm.add(LSTM(units = 100, return_sequences = False))

    model_lstm.add(Dropout(0.2, training=True))
    
    # model_lstm.add(LSTM(units = 50, return_sequences = False))
    
    model_lstm.add(Dense(units = 1, activation='relu'))

    optimizer = Adam(clipvalue=0.5)
    # model_lstm.compile(loss = 'mse', optimizer=optimizer)

    # Compiling
    model_lstm.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    model_lstm.fit(X_train,y_train,epochs=5,batch_size=32, verbose=1)

    LSTM_prediction = model_lstm.predict(X_test)
    LSTM_prediction = sc.inverse_transform(LSTM_prediction)

    # self.model = model_lstm

    return model_lstm, LSTM_prediction

my_LSTM_model, LSTM_prediction = LSTM_model2(X_train, y_train, X_test, sc)

In [ ]:
# LSTM_prediction = sc.inverse_transform(LSTM_prediction)

In [ ]:
def actual_pred_plot(preds):

    actual_pred = pd.DataFrame()

    mask = (ID3.index.year == 2018)
    actual_pred['Actual'] = ID3.loc[mask]['ID3'][0:len(preds)]
    actual_pred['Predicted'] = preds[:, 0]

    from keras.metrics import MeanSquaredError
    m = MeanSquaredError()
    actual_pred['Actual'] = actual_pred['Actual']
    actual_pred['Predicted'] = actual_pred['Predicted']
    m.update_state(np.array(actual_pred['Actual']), np.array(actual_pred['Predicted']))
    
    return (m.result().numpy(), actual_pred)

m, prediction = actual_pred_plot(LSTM_prediction)

In [ ]:
# X = features.drop(labels=['ID3', 'VOL', 'LOAD_F', 'LOAD_FE'], axis=1)
# y = features['ID3']

# X_train, X_test, y_train, y_test = train_test_split(
#     X,
#     y,
#     test_size = 0.3,
#     random_state = 0,
#     shuffle = True)

# X_train.shape, X_test.shape

In [ ]:
# X_train_unscaled = X_train
# X_test_unscaled = X_test

# scaler = StandardScaler()
# scaler.fit(X_train)

# X_train = scaler.transform(X_train_unscaled)

# X_test = scaler.transform(X_test_unscaled)

In [ ]:
# def MLP_model(X_train, y_train, X_test):
#     # create a model
#     from keras.models import Sequential
#     from keras.layers import Dense
#     from keras.optimizers import SGD, Adam
    
#     model_MLP = Sequential()
#     model_MLP.add(Dense(200, activation='relu', kernel_initializer='he_normal', input_shape=(X_train.shape[1],)))
#     model_MLP.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
#     model_MLP.add(Dense(1))

#     optimizer = Adam(clipvalue=0.5)
#     # model_lstm.compile(loss = 'mse', optimizer=optimizer)

#     # Compiling
#     model_MLP.compile(optimizer=optimizer,loss='mean_squared_error')
#     # Fitting to the training set
#     model_MLP.fit(X_train, y_train, epochs = 50, batch_size = 16, verbose = 5)

#     prediction = model_MLP.predict(X_test)
#     # prediction = sc.inverse_transform(prediction)

#     # self.model = model_lstm

#     return model_MLP, prediction

# model, prediction = MLP_model(X_train, y_train, X_test)

In [ ]:
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping
# #Step1. Define the model
# model = Sequential()
# model.add(Dense(16, activation = 'relu', kernel_initializer = 'he_normal', input_shape = (X_train.shape[1],)))
# model.add(Dense(8, activation = 'relu', kernel_initializer = 'he_normal'))
# model.add(Dense(1))
# #Step2. Compile the model
# model.compile(optimizer = 'adam', loss = 'mse', metrics = 'mae')
# #Step3. Fit the model
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,batch_size=64, verbose=5)
# #Step4.1 Evaluate the model
# loss, mae = model.evaluate(X_test, y_test)
# #Step4.2 Plot the learning curve
# plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='val')
# plt.show()

# prediction = model.predict(X_test)

In [ ]:
# actual_pred = pd.DataFrame()

# actual_pred['Actual'] = features['ID3'][0:len(prediction)]
# actual_pred['Predicted'] = prediction[:, 0]

# from keras.metrics import MeanSquaredError
# m = MeanSquaredError()
# actual_pred['Actual'] = actual_pred['Actual'].astype(int)
# actual_pred['Predicted'] = actual_pred['Predicted'].astype(int)
# m.update_state(np.array(actual_pred['Actual']), np.array(actual_pred['Predicted']))

# x, y = actual_pred.index, actual_pred['Actual']

# fig, ax = plt.subplots(figsize = (5 ,5))
# ax.plot(x, y)

# plt.xticks(rotation = 90)

# x, y = actual_pred.index, actual_pred['Predicted']

# fig, ax = plt.subplots(figsize = (5 ,5))
# ax.plot(x, y)

# plt.xticks(rotation = 90);

# m

In [ ]:
####################################################################################################
# Pre

actual_pred = prediction

# actual_pred = pd.DataFrame()

# actual_pred['Actual'] = features['ID3'][0:len(prediction)]
# actual_pred['Predicted'] = prediction[:, 0]

####################################################################################################
# Plot 1

fig, ax = plt.subplots(figsize = (10 ,5))

x, y = actual_pred[0:100].index, actual_pred[0:100]['Actual']

ax.plot(x, y, linewidth=0.5)

####################################################################################################
# Plot 2

x, y = actual_pred[0:100].index, actual_pred[0:100]['Predicted']

ax.plot(x, y, alpha = 0.5)

plt.xticks(rotation = 90);

import plotly.express as px
import plotly.graph_objects as go

actual = pd.DataFrame(['ID3'])
predicted = pd.DataFrame(['ID3'])

actual = pd.DataFrame(actual_pred['Actual'].values)
actual['Type'] = 'Actual'
predicted = pd.DataFrame(actual_pred['Predicted'].values)
predicted['Type'] = 'Predicted'

evaluation = pd.DataFrame(pd.concat([actual, 
                        predicted],
                        axis=0))

fig1 = go.Scatter(      x = actual_pred.index,
                    y = actual_pred['Actual'],
                    # color = evaluation['Type'],
                    # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

fig2 = go.Scatter(      x = actual_pred.index,
                    y = actual_pred['Predicted'],
                    # color = evaluation['Type'],
                    # title = "Log of Appliance Energy Consumption in Wh vs Time"
                    )

data = [fig1, fig2]

fig = go.Figure(data=data)

fig.show()

In [ ]:
n_experiments = 50

test_uncertainty_df = pd.DataFrame()
# test_uncertainty_df['Date'] = testing_df['Date']

for i in range(1, n_experiments + 1):
  experiment_prediction = my_LSTM_model.predict(X_test)
  test_uncertainty_df['ID3_{}'.format(i)] = np.concatenate(sc.inverse_transform(experiment_prediction), axis = 0)
  
  if i % 1 == 0:
    print(f'Experiment: {i}/{n_experiments}')

log_energy_consumption_df = test_uncertainty_df.filter(like='ID3', axis=1)
test_uncertainty_df['ID3_mean'] = log_energy_consumption_df.mean(axis=1)
test_uncertainty_df['ID3_std'] = log_energy_consumption_df.std(axis=1)

test_uncertainty_df = test_uncertainty_df[['ID3_mean', 'ID3_std']]

In [ ]:
test_uncertainty_df['lower_bound'] = test_uncertainty_df['ID3_mean'] - 3*test_uncertainty_df['ID3_std']
test_uncertainty_df['upper_bound'] = test_uncertainty_df['ID3_mean'] + 3*test_uncertainty_df['ID3_std']

test_uncertainty_df.head(20)

In [ ]:
import plotly.graph_objects as go

# datespan = '2018-10-01', '2018-10-04'

test_uncertainty_plot_df = test_uncertainty_df.copy(deep=True)
# test_uncertainty_plot_df = test_uncertainty_plot_df.loc[test_uncertainty_plot_df['Date'].between('2016-08-01', '2018-10-04')]
# truth_uncertainty_plot_df = testing_truth_df.copy(deep=True)
# truth_uncertainty_plot_df = truth_uncertainty_plot_df.loc[testing_truth_df['Date'].between('2016-08-01', '2018-10-04')]

upper_trace = go.Scatter(
    x=actual_pred.index,
    y=test_uncertainty_plot_df['upper_bound'],
    mode='lines',
    fill=None,
    name='99% Upper Confidence Bound'
    )

lower_trace = go.Scatter(
    x=actual_pred.index,
    y=test_uncertainty_plot_df['lower_bound'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 211, 0, 0.5)',
    name='99% Lower Confidence Bound'
    )

# real_trace = go.Scatter(
#     x=truth_uncertainty_plot_df.index,
#     y=truth_uncertainty_plot_df['ID3'],
#     mode='lines',
#     fill=None,
#     name='Real Values'
#     )

# y_test = [item for sublist in y_test for item in sublist]

real_trace = go.Scatter(
    x=actual_pred.index,
    y=actual_pred['Actual'],
    mode='lines',
    fill=None,
    name='Real Values'
    )

mean_trace = go.Scatter(
    x=actual_pred.index,
    y=test_uncertainty_plot_df['ID3_mean'],
    mode='lines',
    fill=None,
    name='Mean Values'
    )

# what_trace = go.Scatter(
#     x=test_uncertainty_plot_df.index,
#     y=np.concatenate(inverse_transform(bayesian_lstm.predict(X_test)), axis=0),
#     mode='lines',
#     fill=None,
#     name='What Values'
#     )

data = [upper_trace, lower_trace, mean_trace, real_trace]

fig = go.Figure(data=data)
fig.update_layout(title='Uncertainty Quantification for Energy Consumption Test Data',
                   xaxis_title='Time',
                   yaxis_title='log_energy_consumption (log Wh)')

fig.layout.font.family = 'SF Mono'

fig.show()

In [ ]:
bounds_df = pd.DataFrame()

# Using 99% confidence bounds
bounds_df['lower_bound'] = test_uncertainty_plot_df['lower_bound']
bounds_df['prediction'] = test_uncertainty_plot_df['log_energy_consumption_mean']
bounds_df['real_value'] = truth_uncertainty_plot_df['log_energy_consumption']
bounds_df['upper_bound'] = test_uncertainty_plot_df['upper_bound']

bounds_df['contained'] = ((bounds_df['real_value'] >= bounds_df['lower_bound']) &
                          (bounds_df['real_value'] <= bounds_df['upper_bound']))

print("Proportion of points contained within 99% confidence interval:", 
      bounds_df['contained'].mean())

In [ ]:
####################################################################################################
# Plot 1

fig, ax = plt.subplots(figsize = (10 ,5))

x, y = actual_pred.index, actual_pred['Actual']

ax.plot(x, y)

In [ ]:
####################################################################################################
# Plot 2

fig, ax = plt.subplots(figsize = (10 ,5))

x, y = actual_pred.index, actual_pred['Predicted']

ax.plot(x, y, alpha = 0.5)

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

smape(actual_pred['Actual'], actual_pred['Predicted'])